In [1]:
import random, numpy as np, pandas as pd, glob as gb, os

from tqdm import tqdm
from tools21cm import read_cbin, XfracFile, DensityFile, VelocityFile, get_dens_redshifts, get_xfrac_redshifts
from tools21cm.irate_file import IonRateFile
from clump_functions import FindNearest, PrintTimeElapsed, SameValuesInArray, SaveMatrix, ReadMatrix

random.seed(0)

In [16]:
i=3
sample_size = 1000
dataset = np.zeros((sample_size*63, 6))

print dataset[i*sample_size:(i+1)*sample_size]

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [82]:
path = '/research/prace/subgrid_sources_500Mpc/300grids/'
fname = path+'20.134-coarsest_wsubgrid_sources.dat.gz'

print 

dens_files = gb.glob(path+'*wsubgrid_sources*')
#print dens_files

redshift = np.sort([float(os.path.split(f)[1].split('-')[0]) for f in gb.glob(path+'*wsubgrid_sources*')])
#return redshifts
print redshifts), np.sort(redshifts).size



[ 6.     6.056  6.113  6.172  6.231  6.292  6.354  6.418  6.483  6.549
  6.617  6.686  6.757  6.83   6.905  6.981  7.059  7.139  7.221  7.305
  7.391  7.48   7.57   7.664  7.76   7.859  7.96   8.064  8.172  8.283
  8.397  8.515  8.636  8.762  8.892  9.026  9.164  9.308  9.457  9.611
  9.771  9.938 10.11  10.29  10.478 10.673 10.877 11.09  11.313 11.546
 11.791 12.048 12.318 12.603 12.903 13.221 13.557 13.914 14.294 14.699
 15.132 15.596 16.095 16.633 17.215 17.848 18.54  19.298 20.134 21.062] 70


In [83]:
dens_path = '/research/prace/sph_smooth_cubepm_130627_12_6912_500Mpc_ext2/global/so/nc300/'
sourc_path = '/research/prace/subgrid_sources_500Mpc/300grids/'
cfact_path = '/research/prace/SubGridPhys/SimClumpMic_180614_500Mpc_nc300-MQL/scat/'
result_path = '/research/prace3/reion/500Mpc_RT/500Mpc_f5_8.2pS_300_stochastic_Cscat/results/'

redshit_in = get_dens_redshifts(dens_path)
redshift_out = get_xfrac_redshifts(result_path)

redshift = SameValuesInArray(SameValuesInArray(redshit_in, redshift_out), np.sort([float(os.path.split(f)[1].split('-')[0]) for f in gb.glob(path+'*wsubgrid_sources*')]))

In [85]:
print redshift

[21.062 20.134 19.298 18.54  17.848 17.215 16.633 16.095 15.596 15.132
 14.699 14.294 13.914 13.557 13.221 12.903 12.603 12.318 12.048 11.791
 11.546 11.313 11.09  10.877 10.673 10.478 10.29  10.11   9.938  9.771
  9.611  9.457  9.308  9.164  9.026  8.892  8.762  8.636  8.515  8.397
  8.283  8.172  8.064  7.96   7.859  7.76   7.664  7.57   7.48   7.391
  7.305  7.221  7.139  7.059  6.981  6.905  6.83   6.757  6.686  6.617
  6.549  6.483  6.418]


In [43]:
import gzip, re

    
def ReadDataSources(fname, save_path='./'):
    with gzip.open(path+'20.134-coarsest_wsubgrid_sources.dat.gz', 'rb') as f:
        lines = f.readlines()
        data2txt = np.zeros(((len(lines)-1)/2, 7))
        for i in range(len(lines)/2):
            idx = 2*i+1
            text = re.sub(r"\s+", " ", lines[idx])+re.sub(r"\s+", " ", lines[idx+1])
            data2txt[i] = np.array(filter(None, text.split(' ')), dtype=float) 
        return data2txt

data = ReadDataSources(fname)

datasources = np.zeros((300,300,300))
for val in data:
    i, j, k = int(val[0])-1, int(val[1])-1, int(val[2])-1
    datasources[i,j,k] = val[4]

In [59]:
def coarse_grid3(arr, noc):
    ''' It coarse a grid from shape=(mesh mesh mesh) to shape=(noc noc noc), a tot of ratio^3 original cells are used to average and create noc cube.
        Parameters:
            * arr (narray)	: 3D array of data
            * noc (int)			: dimension of coarsening of data
    '''
    ratio = int(float(arr.shape[0])/noc)
    coarse_data = np.zeros([noc, noc, noc])  # nocXnocXnoc
    for i in range(noc):
        for j in range(noc):
            for k in range(noc):
                cut = arr[(ratio*i):(ratio*(i+1)), (ratio*j):(ratio*(j+1)), (ratio*k):(ratio*(k+1))]
                coarse_data[i][j][k] = np.mean(cut)
    return coarse_data

datacoarse = coarse_grid(datasources, noc=10)

In [62]:
print datacoarse

[[[0.00836844 0.00916316 0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.0064734  0.         0.         0.
   0.00665722 0.00665722 0.         0.        ]
  [0.         0.01812691 0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.00702683 0.0106028  0.0074026
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.00665722 0.         0.         0.        ]
  [0.01039435 0.         0.         0.         0.         0.
   0.         0.01209843 0.         0.        ]
  [0.         0.         0.00593414 0.         0.01102369 0.00611182
   0.         0.         0.         0.00896106]
  [0.         0.         0.         0.         0.         0.
   0.         0.00702683 0.00611182 0.        ]
  [0

In [3]:
dens_path = '/research/prace/sph_smooth_cubepm_130627_12_6912_500Mpc_ext2/global/so/nc300/'
cfact_path = '/research/prace/SubGridPhys/SimClumpMic_180614_500Mpc_nc300-MQL/scat/'
result_path = '/research/prace3/reion/500Mpc_RT/500Mpc_f5_8.2pS_300_stochastic_Cscat/results/'

redshit_in = get_dens_redshifts(dens_path)
redshift_out = get_xfrac_redshifts(result_path)

redshift = SameValuesInArray(redshit_in, redshift_out)

sample_size = 20000
idx = random.sample(range(300**3), sample_size)

In [4]:
#dataset = np.zeros((sample_size*redshift.size, 8))
dataset = np.zeros((sample_size, 6*redshift.size))

for i in tqdm(range(redshift.size)):
    z = redshift[i]
    
    xi = XfracFile(result_path+'xfrac3d_%.3f.bin' %z).xi.flatten()
    irate = IonRateFile(result_path+'IonRates3_%.3f.bin' %z).irate.flatten()
    ntot = DensityFile(dens_path+'%.3fntot_all.dat' %z).cgs_density.flatten()
    cfact = np.power(10, (read_cbin(cfact_path+'z%.3f_res1.667_scat.dat' %z)).flatten())
    z_arr = np.ones(sample_size)*z
    vx, vy, vz = VelocityFile(dens_path+'%.3fv_all.dat' %z).get_kms_from_density(dens_path+'%.3fntot_all.dat' %z)
    
    # ntot [cgs], vx [km/s], vy [km/s], vz [km/s], cfact, irate [1/s], z, xi
    #dataset[i*sample_size:i*sample_size+sample_size] = np.array([ntot[idx], vx.flatten()[idx], vy.flatten()[idx], vz.flatten()[idx], cfact[idx], irate[idx], z_arr, xi[idx]]).T
    
    dataset[:,i] = ntot[idx]
    dataset[:,i+redshift.size] = vx.flatten()[idx]
    dataset[:,i+2*redshift.size] = vy.flatten()[idx]
    dataset[:,i+3*redshift.size] = vz.flatten()[idx]
    dataset[:,i+4*redshift.size] = cfact[idx]
    dataset[:,i+5*redshift.size] = xi[idx]
    
SaveMatrix('data/reionNNdataset_%d_6x%d.dat' %(sample_size, redshift.size), dataset, size=dataset.shape)

100%|██████████| 63/63 [10:36<00:00, 10.13s/it]


Saving matrix file: data/reionNNdataset_20000_6x63.dat


In [21]:
d = ReadMatrix('data/reionNNdataset_5000_6x63.dat', dimensions=2)
#df = pd.DataFrame(data=d, index=range(315000), columns=['ntot', 'vx', 'vy', 'vz', 'cfact', 'irate', 'z', 'xi'], dtype=np.float32)

labels = np.append(np.append(np.append(np.append(np.append(['ntot_%d' %i for i in range(redshift.size)], ['vx_%d' %i for i in range(redshift.size)]), ['vy_%d' %i for i in range(redshift.size)]), ['vz_%d' %i for i in range(redshift.size)]), ['cfact_%d' %i for i in range(redshift.size)]), ['xi_%d' %i for i in range(redshift.size)])
df = pd.DataFrame(data=d, columns=labels, dtype=np.float32)

df.filter(regex = 'xi_')

In [45]:
df.head()

,ntot_0,ntot_1,ntot_2,ntot_3,ntot_4,ntot_5,ntot_6,ntot_7,ntot_8,ntot_9,...,xi_53,xi_54,xi_55,xi_56,xi_57,xi_58,xi_59,xi_60,xi_61,xi_62
0,2.485373e-30,2.504655e-30,2.523714e-30,2.542529e-30,2.561129e-30,2.579529e-30,2.597773e-30,2.615788e-30,2.633574e-30,2.651127e-30,...,0.998023,0.998077,0.996148,0.996850,0.997102,0.997722,0.998491,0.998774,0.999153,0.999299
1,2.093673e-30,2.097357e-30,2.101131e-30,2.104807e-30,2.108418e-30,2.111932e-30,2.115308e-30,2.118671e-30,2.122087e-30,2.125658e-30,...,0.996450,0.997648,0.997330,0.997847,0.998340,0.998672,0.999242,0.999317,0.999413,0.999444
2,1.990757e-30,1.983050e-30,1.975180e-30,1.967367e-30,1.959374e-30,1.951313e-30,1.943227e-30,1.935165e-30,1.927219e-30,1.919260e-30,...,0.757077,0.888579,0.912188,0.978158,0.994522,0.997339,0.999015,0.999343,0.999517,0.999616
3,2.050291e-30,2.046912e-30,2.043560e-30,2.040165e-30,2.036664e-30,2.033142e-30,2.029593e-30,2.025988e-30,2.022379e-30,2.018758e-30,...,0.704465,0.789056,0.828991,0.918241,0.992750,0.996973,0.998871,0.999306,0.999560,0.999627
4,2.414879e-30,2.423930e-30,2.432320e-30,2.440103e-30,2.447394e-30,2.453689e-30,2.458847e-30,2.463012e-30,2.466045e-30,2.467730e-30,...,0.998933,0.999132,0.999334,0.999496,0.999628,0.999737,0.999759,0.999787,0.999821,0.999837


In [49]:
test = np.log10(df.iloc[:,:-63])
print (test-test.mean(axis=0)).std(axis=0)

ntot_0      0.058355
ntot_1      0.060884
ntot_2      0.063358
ntot_3      0.065781
ntot_4      0.068156
ntot_5      0.070485
ntot_6      0.072773
ntot_7      0.075020
ntot_8      0.077229
ntot_9      0.079403
ntot_10     0.081547
ntot_11     0.083660
ntot_12     0.085746
ntot_13     0.087807
ntot_14     0.089844
ntot_15     0.091858
ntot_16     0.093848
ntot_17     0.095815
ntot_18     0.097759
ntot_19     0.099679
ntot_20     0.101579
ntot_21     0.103455
ntot_22     0.105306
ntot_23     0.107136
ntot_24     0.108942
ntot_25     0.110730
ntot_26     0.112496
ntot_27     0.114241
ntot_28     0.115966
ntot_29     0.117676
              ...   
cfact_33    0.179243
cfact_34    0.185488
cfact_35    0.182745
cfact_36    0.206107
cfact_37    0.205262
cfact_38    0.206279
cfact_39    0.204356
cfact_40    0.208856
cfact_41    0.219241
cfact_42    0.208203
cfact_43    0.211724
cfact_44    0.219574
cfact_45    0.215000
cfact_46    0.222456
cfact_47    0.216273
cfact_48    0.228970
cfact_49    0

In [43]:
(df-df.mean(axis=0))/df.std(axis=0)

,ntot_0,ntot_1,ntot_2,ntot_3,ntot_4,ntot_5,ntot_6,ntot_7,ntot_8,ntot_9,...,xi_53,xi_54,xi_55,xi_56,xi_57,xi_58,xi_59,xi_60,xi_61,xi_62
0,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,0.656102,0.571331,0.482591,0.412843,0.327443,0.246892,0.174993,0.121145,0.080197,0.055162
1,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,0.649535,0.569394,0.488476,0.418548,0.336074,0.255303,0.184211,0.130153,0.086618,0.061668
2,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,...,-0.349535,0.076462,0.064495,0.305950,0.309451,0.243504,0.181425,0.130579,0.089198,0.069355
3,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,...,-0.569121,-0.373324,-0.349800,-0.036710,0.297094,0.240266,0.179661,0.129968,0.090242,0.069864
4,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,0.659900,0.576098,0.498454,0.427979,0.345056,0.264720,0.190566,0.137937,0.096706,0.079258
5,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,-0.095169,0.494628,0.468116,0.406999,0.331802,0.252589,0.175530,0.119839,0.076471,0.052834
6,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,...,0.662442,0.578423,0.500400,0.429518,0.346382,0.265794,0.192042,0.139685,0.098851,0.082737
7,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,...,0.645899,0.565410,0.486100,0.416940,0.335485,0.258534,0.190828,0.138635,0.097934,0.081428
8,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,...,0.656733,0.575593,0.498572,0.428055,0.344934,0.264311,0.191154,0.138525,0.097221,0.079863
9,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,0.618049,0.546085,0.481142,0.415867,0.340626,0.261010,0.188270,0.135314,0.093532,0.073866


In [41]:
print df.std(axis=0)

ntot_0     0.000000
ntot_1     0.000000
ntot_2     0.000000
ntot_3     0.000000
ntot_4     0.000000
ntot_5     0.000000
ntot_6     0.000000
ntot_7     0.000000
ntot_8     0.000000
ntot_9     0.000000
ntot_10    0.000000
ntot_11    0.000000
ntot_12    0.000000
ntot_13    0.000000
ntot_14    0.000000
ntot_15    0.000000
ntot_16    0.000000
ntot_17    0.000000
ntot_18    0.000000
ntot_19    0.000000
ntot_20    0.000000
ntot_21    0.000000
ntot_22    0.000000
ntot_23    0.000000
ntot_24    0.000000
ntot_25    0.000000
ntot_26    0.000000
ntot_27    0.000000
ntot_28    0.000000
ntot_29    0.000000
             ...   
xi_33      0.226983
xi_34      0.234076
xi_35      0.241141
xi_36      0.248305
xi_37      0.254519
xi_38      0.260329
xi_39      0.264819
xi_40      0.269958
xi_41      0.273918
xi_42      0.278109
xi_43      0.280558
xi_44      0.282179
xi_45      0.282664
xi_46      0.283494
xi_47      0.283032
xi_48      0.280328
xi_49      0.276152
xi_50      0.270339
xi_51      0.264728


In [5]:
#print df.loc[:, 'irate'].min()
#df.loc[:, 'irate'] = np.log10(df.loc[:, 'irate']).replace(-np.inf, -100)
#df.loc[:, 'irate'] = df.loc[:, 'irate']/1e-12
print np.log10(df.loc[:, 'irate']).min() # .replace(-np.inf, -100)
print np.log10(df.loc[:, 'irate']/1e-12).min()
#print df.loc[:, 'irate'].min()


-inf
-inf
ntot     2.094204e-30
vx       2.546950e-01
vy      -3.534850e-02
vz       7.449942e-01
cfact    1.551256e+01
irate    2.054028e-13
z        1.067109e+01
xi       3.900851e-01
dtype: float32
